In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
def load_data():
    train = pd.read_csv("/kaggle/input/titanic/train.csv")
    train.name = 'train'
    test = pd.read_csv("/kaggle/input/titanic/test.csv")
    test.name = 'test'
    return train, test

In [3]:
train_data, test_data = load_data()
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
def clean_data(df):
    name = df.name
    df['Sex'] = df['Sex'].replace({"male": 0, "female": 1})
    df = pd.get_dummies(df, columns=['Embarked'])
    df = df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)
    if name != 'test':
        X_train = df.drop('Survived', axis = 1)
        X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())
        y_train = df['Survived']
        return X_train, y_train
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    return df

In [32]:
X_train, y_train = clean_data(train_data)
X_test = clean_data(test_data)

In [33]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,0,22.0,1,0,7.2500,False,False,True
1,1,1,38.0,1,0,71.2833,True,False,False
2,3,1,26.0,0,0,7.9250,False,False,True
3,1,1,35.0,1,0,53.1000,False,False,True
4,3,0,35.0,0,0,8.0500,False,False,True


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(), LogisticRegression(random_state = 42 ,max_iter = 1000))
pipe.fit(X_train, y_train)
train_preds = pipe.predict(X_train)

In [35]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_train, train_preds))
print(accuracy_score(y_train, train_preds))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       549
           1       0.76      0.70      0.73       342

    accuracy                           0.80       891
   macro avg       0.79      0.78      0.79       891
weighted avg       0.80      0.80      0.80       891

0.8002244668911336


In [37]:
y_pred = pipe.predict(X_test)
output = np.c_[test_data["PassengerId"].to_numpy(), y_pred].astype(int)
output = pd.DataFrame(output, columns = ['PassengerId', 'Survived'])
output.to_csv("sklearn_logistic_submission.csv", index = False)

1